In [2]:
import json
import os

#  
from collections import Counter 

In [9]:
# list of concept
concept_list = [ 
"jpcrp_cor:NetSalesSummaryOfBusinessResults",
"jpcrp_cor:RevenueKeyFinancialData",
"jpcrp_cor:OperatingRevenue1SummaryOfBusinessResults",
"jpcrp_cor:OperatingRevenue2SummaryOfBusinessResults",
"jpcrp_cor:GrossOperatingRevenueSummaryOfBusinessResults",
"jpcrp_cor:OrdinaryIncomeSummaryOfBusinessResults",
"jpcrp_cor:NetPremiumsWrittenSummaryOfBusinessResultsINS",
"jpcrp_cor:OrdinaryIncomeLossSummaryOfBusinessResults",
"jpcrp_cor:ProfitLossAttributableToOwnersOfParentSummaryOfBusinessResults",
"jpcrp_cor:ComprehensiveIncomeSummaryOfBusinessResults",
"jpcrp_cor:NetAssetsSummaryOfBusinessResults",
"jpcrp_cor:TotalAssetsSummaryOfBusinessResults",
"jpcrp_cor:RateOfReturnOnEquitySummaryOfBusinessResults",
"jpcrp_cor:PriceEarningsRatioSummaryOfBusinessResults",
"jpcrp_cor:NetCashProvidedByUsedInOperatingActivitiesSummaryOfBusinessResults",
"jpcrp_cor:NetCashProvidedByUsedInInvestingActivitiesSummaryOfBusinessResults",
"jpcrp_cor:NetCashProvidedByUsedInFinancingActivitiesSummaryOfBusinessResults",
"jpcrp_cor:CashAndCashEquivalentsSummaryOfBusinessResults",
"jpcrp_cor:NumberOfEmployees",
"jpcrp_cor:RevenueIFRSSummaryOfBusinessResults",
"jpcrp_cor:ProfitLossBeforeTaxIFRSSummaryOfBusinessResults",
"jpcrp_cor:ProfitLossFromContinuingOperationsIFRSSummaryOfBusinessResults",
"jpcrp_cor:ProfitLossIFRSSummaryOfBusinessResults",
"jpcrp_cor:ProfitLossAttributableToOwnersOfParentIFRSSummaryOfBusinessResults",
"jpcrp_cor:ComprehensiveIncomeIFRSSummaryOfBusinessResults",
"jpcrp_cor:ComprehensiveIncomeAttributableToOwnersOfParentIFRSSummaryOfBusinessResults",
"jpcrp_cor:EquityAttributableToOwnersOfParentIFRSSummaryOfBusinessResults",
"jpcrp_cor:TotalAssetsIFRSSummaryOfBusinessResults",
"jpcrp_cor:PriceEarningsRatioIFRSSummaryOfBusinessResults",
"jpcrp_cor:CashFlowsFromUsedInOperatingActivitiesIFRSSummaryOfBusinessResults",
"jpcrp_cor:CashFlowsFromUsedInInvestingActivitiesIFRSSummaryOfBusinessResults",
"jpcrp_cor:CashFlowsFromUsedInFinancingActivitiesIFRSSummaryOfBusinessResults",
"jpcrp_cor:CashAndCashEquivalentsIFRSSummaryOfBusinessResults"
]

In [38]:
# get the statistics 

prefix_concepts = dict()
contextre_dict = dict()

for id_name in os.listdir("/home/nedo/data/edinet/unzip"):
    prefix_concepts[id_name] = []
    contextre_dict[id_name] = []
    input_dir = os.path.join("/home/okajima/edinet/edinet/ma/data/output/mna/",id_name)

   
    for json_file in os.listdir(input_dir):

        with open(input_dir+'/'+json_file) as f:
            all_objs = json.load(f)
            for i in all_objs:
                try: 
                    prefix_concepts[id_name].append(i["attributes"]["name"])
                    contextre_dict[id_name].append(i["attributes"]["contextRef"])
                except:
                    continue

In [36]:
# get the statistis of prefix
prefix_concepts1 = []

for k in prefix_concepts:
    prefix_concepts1.extend(prefix_concepts[k])

# focus only on jpcrp_cor
prefix_concepts2 = [i for i in prefix_concepts1 if i[:9] == 'jpcrp_cor']

# get the frequency
dict_freq = dict(Counter(prefix_concepts2))

# filter out the concepts < 1000
dict_freq2 = dict()
for i in concept_list:
    try: 
        if dict_freq[i] > 1000:
            dict_freq2[i] = dict_freq[i]
    except:
        pass

In [47]:
len(dict_freq2) # 26 concpets are selected

dict_res = dict()

for id_name in os.listdir("/home/nedo/data/edinet/unzip"):
    dict_res[id_name] = {}
    input_dir = os.path.join("/home/okajima/edinet/edinet/ma/data/output/mna/",id_name)

    ct = 0
    for json_file in os.listdir(input_dir):

        with open(input_dir+'/'+json_file) as f:
            all_objs = json.load(f)
            for i in all_objs:
                for j in list(dict_freq2.keys()):
                    try: 
                        if i["attributes"]["name"] == j:
                            dict_res[id_name][ct] = i
                            ct += 1
                    except:
                        continue


In [49]:
with open('./output/fileid_year', 'r') as f:
    fileid_year = json.load(f)


In [65]:
# new dictionary including year information
# add up REAL year information (according to ending YEAR of file)
dict_res2 = dict()

for id_name in dict_res.keys():
    # initialize the dict
    dict_res2[id_name] = dict_res[id_name]
    # get current year
    current_year = int(fileid_year[id_name]['currentEnd'][:4])

    for item_k in dict_res[id_name].keys():
        item = dict_res[id_name][item_k]
        # print(item['attributes']['contextRef'][:6])
        # calculate the REAL year for this item
        if item['attributes']['contextRef'] in ['Prior4YearInstant','Prior4YearDuration'] :
            calculate_year = current_year - 4
        if item['attributes']['contextRef'] in ['Prior3YearInstant','Prior3YearDuration']:
            calculate_year = current_year - 3
        if item['attributes']['contextRef'] in ['Prior2YearInstant','Prior2YearDuration']:
            calculate_year = current_year - 2
        if item['attributes']['contextRef'] in ['Prior1YearInstant','Prior1YearDuration']:
            calculate_year = current_year - 1
        if item['attributes']['contextRef'] in ['CurrentYearInstant','CurrentYearDuration']:    
            calculate_year = current_year
        else:
            calculate_year = 0
        
        # store year info into item
        dict_res2[id_name][item_k]['year'] = int(calculate_year)




In [66]:
dict_res2[id_name]

{0: {'type': 'nonFraction',
  'attributes': {'contextRef': 'Prior4YearDuration',
   'decimals': '-6',
   'scale': '6',
   'format': 'ixt:numdotdecimal',
   'name': 'jpcrp_cor:NetSalesSummaryOfBusinessResults',
   'unitRef': 'JPY'},
  'taxonomy': 'jpcrp_cor',
  'concept': 'NetSalesSummaryOfBusinessResults',
  'value': '102,903',
  'year': 0},
 1: {'type': 'nonFraction',
  'attributes': {'contextRef': 'Prior3YearDuration',
   'decimals': '-6',
   'scale': '6',
   'format': 'ixt:numdotdecimal',
   'name': 'jpcrp_cor:NetSalesSummaryOfBusinessResults',
   'unitRef': 'JPY'},
  'taxonomy': 'jpcrp_cor',
  'concept': 'NetSalesSummaryOfBusinessResults',
  'value': '101,601',
  'year': 0},
 2: {'type': 'nonFraction',
  'attributes': {'contextRef': 'Prior2YearDuration',
   'decimals': '-6',
   'scale': '6',
   'format': 'ixt:numdotdecimal',
   'name': 'jpcrp_cor:NetSalesSummaryOfBusinessResults',
   'unitRef': 'JPY'},
  'taxonomy': 'jpcrp_cor',
  'concept': 'NetSalesSummaryOfBusinessResults',
  'v

In [46]:
### add up REAL year information (according to ending YEAR of file)

for item in dict_res:
    



['jpcrp_cor:NetSalesSummaryOfBusinessResults',
 'jpcrp_cor:OperatingRevenue1SummaryOfBusinessResults',
 'jpcrp_cor:OrdinaryIncomeSummaryOfBusinessResults',
 'jpcrp_cor:OrdinaryIncomeLossSummaryOfBusinessResults',
 'jpcrp_cor:ProfitLossAttributableToOwnersOfParentSummaryOfBusinessResults',
 'jpcrp_cor:ComprehensiveIncomeSummaryOfBusinessResults',
 'jpcrp_cor:NetAssetsSummaryOfBusinessResults',
 'jpcrp_cor:TotalAssetsSummaryOfBusinessResults',
 'jpcrp_cor:RateOfReturnOnEquitySummaryOfBusinessResults',
 'jpcrp_cor:PriceEarningsRatioSummaryOfBusinessResults',
 'jpcrp_cor:NetCashProvidedByUsedInOperatingActivitiesSummaryOfBusinessResults',
 'jpcrp_cor:NetCashProvidedByUsedInInvestingActivitiesSummaryOfBusinessResults',
 'jpcrp_cor:NetCashProvidedByUsedInFinancingActivitiesSummaryOfBusinessResults',
 'jpcrp_cor:CashAndCashEquivalentsSummaryOfBusinessResults',
 'jpcrp_cor:NumberOfEmployees',
 'jpcrp_cor:RevenueIFRSSummaryOfBusinessResults',
 'jpcrp_cor:ProfitLossBeforeTaxIFRSSummaryOfBusiness

In [40]:


# get the statistis of year indicators
contextre_dict1 = []

for k in contextre_dict:
    contextre_dict1.extend(contextre_dict[k])

Counter(contextre_dict1)

Counter({'Prior4YearDuration': 109581,
         'Prior3YearDuration': 111502,
         'Prior2YearDuration': 112781,
         'Prior1YearDuration': 1208404,
         'CurrentYearDuration': 1370314,
         'Prior4YearInstant': 67691,
         'Prior3YearInstant': 68834,
         'Prior2YearInstant': 94300,
         'Prior1YearInstant': 708982,
         'CurrentYearInstant': 806439,
         'Prior4YearDuration_NonConsolidatedMember': 110774,
         'Prior3YearDuration_NonConsolidatedMember': 111283,
         'Prior2YearDuration_NonConsolidatedMember': 111653,
         'Prior1YearDuration_NonConsolidatedMember': 619386,
         'CurrentYearDuration_NonConsolidatedMember': 633389,
         'Prior4YearInstant_NonConsolidatedMember': 96599,
         'Prior3YearInstant_NonConsolidatedMember': 97047,
         'Prior2YearInstant_NonConsolidatedMember': 109402,
         'Prior1YearInstant_NonConsolidatedMember': 845047,
         'CurrentYearInstant_NonConsolidatedMember': 870674,
         

In [51]:
id_name

'E05607_S100FA48'